In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yaml

config = yaml.load(open('../config/full_images_to_hist_config.yaml', 'r'), Loader=yaml.FullLoader)
n_bins = config['clusterization_config']['n_clusters']

In [ ]:
# evaluating ranking method
def evaluate_query(df, image_query, measure="cosine", normalized=True, feature_type = "histogram"):
    # obtener la clase de la imagen
    image_class = SIFT.get_class(image_query)
    print(image_class)
    # calcular similudes
    df = query_image(df, image_query, measure=measure, feature_type = feature_type)
    # obtener los indices de las imagenes de la misma clase
    indexes = df[df["class"] == image_class].index.values.tolist()
    print(indexes)
    n_rel = len(indexes) - 1

    if normalized:
        N = len(df)
        return 1/(n_rel*N) * (sum(indexes) - (n_rel*(n_rel+1))/2)

    return 1/n_rel * sum(indexes)

image_example = df["image_name"][0]
evaluate_query(df, image_example, measure="euclidean", normalized=True)

In [ ]:
from src.CNN.CNN_utils import  CNN_method, deep_rank_model
import tqdm

import os
from PIL import Image

def launch_CNN():
    model = deep_rank_model()
    model.load_weights(input())

    def load_single(path):
        image = Image.open(os.path.join(path))
        return image

    CNN_method(load_single(f'/root/labint/scripts/LabInt/data/GPR1200/images/{df["image_name"][0]}'), model)

    def apply_cnn(image, CNN_method):
        try:
            return CNN_method(image, model)
        except:
            return np.zeros(4096)

    df["features_CNN"] = tqdm.tqdm(df["image_name"].apply(lambda x: apply_cnn(load_single(f'/root/labint/scripts/LabInt/data/GPR1200/images/{x}'), CNN_method), model))
 

In [ ]:
def plot_images(df_, index):
    plt.figure(figsize=(10, 10))
    plt.imshow(Image.open(f'{path_to_images}/{df_query["image_name"][i+j]}'))
    plt.show()

fig,ax = plt.subplots(nrows=2,ncols=5,figsize=(15, 6))

fig.suptitle("Top Matching Images", fontsize=15)
ax[0][0].set_title(f"Consulta")
x = 0
for j in range(0, 2):

    for i in range(0,5): 
        ax[j][i].set_axis_off()
        ax[j][i].imshow(Image.open(f'{path_to_images}/{df_query["image_name"][x]}'))
        if x > 0:
            ax[j][i].set_title(f"Match {x-1}")
        x +=1

In [ ]:
def get_hist(value, separator = ','):
    hist = str(value).strip().replace('[','').replace(']','').replace("\n","").split(separator)
    hist = [i for i in hist if i != '']
    hist = np.array(hist, dtype=np.float32)
    return hist